In [1]:
import json
import pickle
from pathlib import Path
from typing import List

import cv2 as cv
import numpy as np
from sklearn.mixture import BayesianGaussianMixture

In [2]:
laser_samples_path = Path('laser_data')
pkl_model_path = Path('model.pkl')
json_model_path = Path('model.json')

In [3]:
laser_samples = laser_samples_path.rglob('*.jpg', case_sensitive=False)

In [4]:
imgs = [cv.imread(path) for path in laser_samples]

In [5]:
hsv_images: List[cv.Mat] = [cv.cvtColor(img, cv.COLOR_BGR2HSV) for img in imgs]

In [6]:
flattened_image = np.empty((0, 3), dtype=np.float32)
for img in hsv_images:
    flattened_image = np.vstack((flattened_image, img.reshape(-1, 3).astype(np.float32)))

In [7]:
gmm = BayesianGaussianMixture(
    n_components=10,
    max_iter=200,
    covariance_type='full'
)
gmm.fit(flattened_image)

BayesianGaussianMixture(max_iter=200, n_components=10)

In [8]:
with open(pkl_model_path, 'wb') as handle:
    pickle.dump(gmm, handle)

In [9]:
len(gmm.means_)

10

In [10]:
for cov in gmm.covariances_:
    print(cov)

[[ 1.01268702e+01 -1.24852124e-01 -2.67166623e+00]
 [-1.24852124e-01  5.41744811e+00 -2.36576107e+01]
 [-2.67166623e+00 -2.36576107e+01  3.90119771e+03]]
[[1051.5671811    63.55685238 -102.42461619]
 [  63.55685238  387.07326981 -170.00720419]
 [-102.42461619 -170.00720419 1276.45783636]]
[[1051.56753436   63.55696793 -102.42472276]
 [  63.55696793  387.07327278 -170.00726698]
 [-102.42472276 -170.00726698 1276.45749801]]
[[  65.5724329    30.38455775    7.44855838]
 [  30.38455775  404.17083821  -58.71679204]
 [   7.44855838  -58.71679204 1426.36671126]]
[[1051.56857249   63.55730752 -102.42503592]
 [  63.55730752  387.07328149 -170.00745148]
 [-102.42503592 -170.00745148 1276.4565037 ]]
[[1051.56924726   63.55752824 -102.42523947]
 [  63.55752824  387.07328714 -170.00757141]
 [-102.42523947 -170.00757141 1276.45585742]]
[[1051.56990823   63.55774393 -102.42543861]
 [  63.55774393  387.07329147 -170.00768872]
 [-102.42543861 -170.00768872 1276.45522794]]
[[1051.57055698   63.55795667 

In [11]:
with open(json_model_path, 'w', encoding='utf-8') as handle:
    json.dump({
        'weights': gmm.weights_.tolist(),
        'means': gmm.means_.tolist(),
        'covariances': gmm.covariances_.tolist()
    }, handle)